# Getting available actions

**Note** this notebook requres that there are pending actions in the database. So go create some first. 

Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
but only one discussed here. 
* ~~web interaction that creates jobs~~
* **azure functions that resolve actions**

In [1]:
import sys
import numpy as np
import pandas as pd


sys.path.append("../..")
sys.path.append("..")
import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()


from app.connectors import cmdb_graph
from app.objects import time as t


c = cmdb_graph.CosmosdbClient()


executing local windows deployment


In [2]:
time = t.Time(c)
time.get_current_UTU()
time

< time at: 2023-07-20T04:38:19.297013+00:00 UTU:28298 >

In [3]:
time.get_global_actions()
actions_df = pd.DataFrame(time.actions)
actions_df

,action,job,agent
0,"{'type': 'build_infrastructure', 'applies_to':...","{'status': 'pending', 'username': 'Billmanserv...","{'isIdle': 'false', 'name': 'Serzeampro Wil', ..."


All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


In [4]:
for i in actions_df.index:
    action = t.Action(c,actions_df.loc[i])
    print(action)
    

< (Serzeampro Wil: 1772981049949) -takingAction:28301-> (build infrastructure) >


## Validate that the action can be completed

In [5]:
action

< (Serzeampro Wil: 1772981049949) -takingAction:28301-> (build infrastructure) >

In [6]:
action.job

{'status': 'pending',
 'username': 'Billmanserver',
 'name': 'takingAction',
 'weight': '28301',
 'actionType': 'build_infrastructure'}

In [7]:
action.validate_action_time(time)

False

the validation returns true/false and simply confirms that the weight is larger than the current UTC time. 

## Complete the action

In [8]:
action.action.get('augments_self_properties')

'{wealth: -0.1, industry: 0.1}'

In the table below `og` is the origional value, the updated is what the agent's value will change to. 

In [9]:
pd.DataFrame([action.agent, action.resolve_augments_self_properties()], index = ['og','updated'])

,isIdle,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,username,objtype
og,false,Serzeampro Wil,1772981049949,0.424,0.469,0.647,0.668,0.7,4873232607770,0.6575,0.5632,0.394,Billmanserver,pop
updated,false,Serzeampro Wil,1772981049949,0.424,0.469,0.647,0.668,0.7,4873232607770,0.7575,0.4632,0.394,Billmanserver,pop


In [10]:
action.add_updates_to_c(time)
action.c.stack

["g.V().has('objid','1772981049949').property('wealth',0.5632).property('industry',0.6575)",
 "g.V().has('objid','1772981049949').outE('takingAction').has('actionType','build_infrastructure').has('weight','28301').property('status','resolved')",
 "g.V().has('objid','1772981049949').property('isIdle','true')"]

In [11]:
action.data

{'nodes': [{'objid': '2966917945635',
   'name': 'job',
   'label': 'event',
   'text': 'The pop (Serzeampro Wil) has completed build_infrastructure',
   'visibleTo': 'Billmanserver',
   'time': 28298,
   'username': 'event'}],
 'edges': [{'node1': '1772981049949',
   'node2': '2966917945635',
   'label': 'completed'}]}

Veryify that actions are resolved or not

In [12]:
action.resolve_action()

In [14]:
time.get_global_actions()
actions_df = pd.DataFrame(time.actions)
actions_df

""


If the `actions_df` is empty then the action has resolved. 